In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오기

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

# 컬럼명 바꾸기

In [3]:
train = train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철',
                     '도보 10분거리 내 버스정류장 수':'버스'})

test = test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철',
                     '도보 10분거리 내 버스정류장 수':'버스'})

In [13]:
train['지역'].value_counts()

경기도        621
경상남도       369
부산광역시      328
대전광역시      266
강원도        187
충청북도       163
충청남도       158
전라남도       151
광주광역시      141
경상북도       119
전라북도       109
대구광역시      102
제주특별자치도     91
서울특별시       64
세종특별자치시     47
울산광역시       36
Name: 지역, dtype: int64

In [12]:
train['임대건물구분'].value_counts()

아파트    2383
상가      569
Name: 임대건물구분, dtype: int64

# 결측치 처리

In [6]:
print("-------- train.csv --------")
print(train.isnull().sum())
print("-------- test.csv --------")
print(test.isnull().sum())

-------- train.csv --------
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       569
임대료         569
지하철         211
버스            4
단지내주차면수       0
등록차량수         0
dtype: int64
-------- test.csv --------
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64


In [7]:
# train 데이터
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)
train['지하철'] = train['지하철'].fillna(0)
train['버스'] = train['버스'].fillna(0)

# test 데이터
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['지하철'] = test['지하철'].fillna(0)

test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

In [8]:
print("-------- train.csv --------")
print(train.isnull().sum())
print("-------- test.csv --------")
print(test.isnull().sum())

-------- train.csv --------
단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
등록차량수       0
dtype: int64
-------- test.csv --------
단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
dtype: int64


# '-' 데이터 처리

In [9]:
# 임대료, 임대보증금 데이터에 - 값이 있어서 이를 0으로 변환
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
train.loc[train.임대료=='-', '임대료'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)

test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['임대보증금'] = test['임대보증금'].astype(float)
test['임대료'] = test['임대료'].astype(float)

# 지하철 Y or N로 변경

In [10]:
train['지하철'] = np.where(train['지하철'] == 0, 0, 1)
test['지하철'] = np.where(test['지하철'] == 0, 0, 1)

# 임대건물구분 컬럼 처리

In [11]:
# 임대건물구분에서 7개의 아파트는 상가로 예상이됨 이를 처리 함.
# 임대건물구분이 상가인 경우는 전용면적별세대수도 1이므로 이를 처리 함.

idx = train[(train['임대건물구분']=='아파트') & (train['자격유형']=='D')]['전용면적별세대수'].index
train.loc[idx, '전용면적별세대수'] = 1
train.loc[idx, '임대건물구분'] = '상가'

# 지역 데이터를 숫자로 매핑

In [19]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
train['지역'] = train['지역'].map(local_map)

local_map = {}
for i, loc in enumerate(test['지역'].unique()):
    local_map[loc] = i
    
test['지역'] = test['지역'].map(local_map)

# 전용면적을 상/하한 적용

In [11]:
# 전용면적을 5의 배수로 변경 후 상/하한 적용
# 하한 값 15, 상한 값 100을 적용

train['전용면적'] = np.round(train['전용면적'], 0)
train['전용면적'] = train['전용면적']//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

test['전용면적'] = np.round(test['전용면적'], 0)
test['전용면적'] = test['전용면적']//5*5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

# 학습 데이터 생성

In [12]:
train_data = train[['총세대수','전용면적','전용면적별세대수','공가수','임대보증금','임대료','단지내주차면수','등록차량수']]
test_data = test[['총세대수','전용면적','전용면적별세대수','공가수','임대보증금','임대료','단지내주차면수']]
print("train 데이터:",train_data.shape)
print("test 데이터:",test_data.shape)

train 데이터: (2952, 8)
test 데이터: (1022, 7)


In [13]:
# 데이터 target 설정

x_train = train_data.iloc[:,:7]
y_train = train_data.iloc[:,-1]
x_test = test_data
print("x_train 데이터:",x_train.shape)
print("y_train 데이터:",y_train.shape)
print("x_train 데이터:",x_test.shape)

x_train 데이터: (2952, 7)
y_train 데이터: (2952,)
x_train 데이터: (1022, 7)


# 모델링

In [14]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_jobs=-1, random_state=42)

forest.fit(x_train, y_train)

pred = forest.predict(x_test)

In [15]:
import xgboost

xgb = xgboost.XGBRegressor()

xgb.fit(x_train, y_train)

pred = xgb.predict(x_test)

# 예측 데이터셋 생성

In [16]:
df = []
df = pd.DataFrame(df)
df['code'] = test['단지코드']
df['num'] = pred
sub = df.groupby(['code'],sort=False,as_index=False).mean()

In [17]:
sub.to_csv('xgb.csv', index=False)
sub.head()

,code,num
0,C1072,649.207397
1,C1128,1191.849976
2,C1456,638.613586
3,C1840,467.848450
4,C1332,1009.499634


# 모델 평가

In [18]:
from sklearn.model_selection import train_test_split

x = train_data.iloc[:,:7]
y = train_data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0)

from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(random_state=0)

forest.fit(x_train, y_train)

pred = forest.predict(x_test)

from sklearn.metrics import mean_absolute_error
print("mae :",mean_absolute_error(y_test, pred))

from sklearn.metrics import r2_score
print("r2_score :",r2_score(y_test, pred))

mae : 28.853726287262873
r2_score : 0.9783652648951725


In [19]:
from sklearn.model_selection import train_test_split

x = train_data.iloc[:,:7]
y = train_data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0)

import xgboost

xgb = xgboost.XGBRegressor(random_state=0)

xgb.fit(x_train, y_train)

pred = xgb.predict(x_test)

from sklearn.metrics import mean_absolute_error
print("mae :",mean_absolute_error(y_test, pred))

from sklearn.metrics import r2_score
print("r2_score :",r2_score(y_test, pred))

mae : 29.864864125807433
r2_score : 0.9804879974711456
